Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a [kaggle competition](https://www.kaggle.com/c/whiskey-201911/) We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

We are going to run increasingly sophisticated classification models on our whisky reviews in parts 1, 2, and 3. For each of parts 1, 2, and 3, submit your best model's results to the Kaggle competition to measure `generalization accuracy` -- i.e. how well the model performs on new data.

##1. Classifier based on TfIdf vectorization of reviews

### Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### 1.0 Setup

#### 1.0.1 Get spacy and restart runtime

In [14]:
#YOUR CODE HERE
# !python3 -m spacy download en_core_web_md

     |████████████████████████████████| 45.4 MB 10.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


#### 1.0.2 import necessary packages, load spacy

In [15]:
import pandas as pd
import re

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
import spacy

Load `spacy`

In [36]:
#YOUR CODE HERE
nlp = spacy.load('en_core_web_lg')

#### 1.0.3 Load Kaggle Whisky Competition Data

In [64]:
# !!!!! You may need to change the path !!!!!
# You can download these datasets from the Kaggle in-class 
# competition for your cohort. 
 
train = pd.read_csv('whiskey/train.csv', index_col='id')
test = pd.read_csv('whiskey/test.csv')

In [65]:
test

,id,description
0,955,"Think carnival aromas—the good ones, anyway—me..."
1,3532,"A blend of three bourbons, between 6 and 12 ye..."
2,1390,"The nose is focused on cereal, hints of fresh ..."
3,1024,Swiss-based Chapter 7 released this 19 year ol...
4,1902,Valkyrie replaces the current Dark Origins exp...
...,...,...
283,1834,"Aged in a sherry cask, and the sherry is clean..."
284,2848,There have been some legendary Hart Brothers b...
285,3874,"Aromas hint at a bitterly astringent, almost b..."
286,2377,"Think corn: roasted, canned, creamed, mashed, ..."


### 1.1 Clean Text

In [41]:
def clean_doc(text):
  # COMPLETE THE CODE IN THIS CELL
  # remove new line characters
  text = text.replace('\\n', ' ')
  # remove numbers for the text
  text = re.sub('[^a-z A-Z]', '', text)
  # remove multiple white spaces
  text = re.sub('[ ]{2,}', ' ', text)

  # case normalize and strip extra white spaces on the far left and right hand side
  text = text.lower().strip()
  return text

train['description'] = train['description'].apply(clean_doc)
test['description'] = test['description'].apply(clean_doc)


In [43]:
train['description']

id
1       a marriage of and year old bourbons a mature y...
2       there have been some legendary bowmores from t...
3       this bottling celebrates master distiller park...
4       what impresses me most is how this whisky evol...
9       a caramelladen fruit bouquet followed by unend...
                              ...                        
4146    earthy fleshy notes with brooding grape notes ...
4153    with its overt floral perfume notes and the sc...
4154    an unaged whiskey from carroll county iowa wit...
4155    fiery peat kiln smoke tar and ripe barley on t...
4157    although its not on the label cavalry uses the...
Name: description, Length: 2586, dtype: object

### 1.2 Split training data into Feature Matrix and Target Vector

In [49]:
target = 'category'
# COMPLETE THE CODE IN THIS CELL
y = train[target]
X = train['description']

In [45]:
print(X)

                                            description
id                                                     
1     a marriage of and year old bourbons a mature y...
2     there have been some legendary bowmores from t...
3     this bottling celebrates master distiller park...
4     what impresses me most is how this whisky evol...
9     a caramelladen fruit bouquet followed by unend...
...                                                 ...
4146  earthy fleshy notes with brooding grape notes ...
4153  with its overt floral perfume notes and the sc...
4154  an unaged whiskey from carroll county iowa wit...
4155  fiery peat kiln smoke tar and ripe barley on t...
4157  although its not on the label cavalry uses the...

[2586 rows x 1 columns]


### 1.3 Define Pipeline Components
We can try`RandomForestClassifier()`,  `GradientBoostingClassifier()` from the `sklearn` library, and `XGBClassifier()` from the `xgboost` library.

In [46]:
# limiting max_features to 500 to speed up training on Colab.
# COMPLETE THE CODE IN THIS CELL
vect = TfidfVectorizer(stop_words='english', tokenizer=None)
clf = RandomForestClassifier(random_state=42)

pipe = Pipeline([('vect', vect), ('clf', clf)])

### 1.4 Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [47]:
print(len(X))
print(len(y))

2586
2586


In [50]:
# COMPLETE THE CODE IN THIS CELL
# Parameters to search in dictionary 
parameters = {
    'vect__max_df': [.75], #[x/100 for x in range(75, 101, 25)],
    'vect__min_df': [10], #[x for x in range(6, 11, 2)],
    'vect__max_features': [600], #[x for x in range(500, 1001, 100)],
    # 'vect__ngram_range': [(1, x+1) for x in range(3)],
    'clf__n_estimators': [100], #[x for x in range(20, 101, 20)],
    'clf__max_depth': [25] #[x for x in range(5, 26, 5)]
}

# Implement a grid search with cross-validation
grid_search = GridSearchCV(pipe, parameters, n_jobs=-2, cv=3, verbose=1)
grid_search.fit(X, y)

# Display the best score from the grid search
print(grid_search.best_score_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:    2.1s finished


0.8708430007733953


In [51]:
# Display the best parameters from the grid search
print(grid_search.best_params_)
print(grid_search.best_score_)

{'clf__max_depth': 25, 'clf__n_estimators': 100, 'vect__max_df': 0.75, 'vect__max_features': 600, 'vect__min_df': 10}
0.8708430007733953


### 1.5 Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so only submit when your predicted test accuracy is the highest you can make it. For this competition the max daily submissions are capped at **20**.  The submission file is made from the results of running your best model on the test data set, for which we don't get the targets.

In [66]:
# COMPLETE THE CODE IN THIS CELL
# Predictions on **test** sample
pred = grid_search.predict(test['description'])

In [67]:
pred

array([2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 4, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1,
       1, 1, 4, 1, 1, 1, 3, 1, 4, 2, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 4,
       2, 1, 1, 1, 1, 3, 1, 1, 4, 1, 3, 2, 1, 1, 1, 1, 2, 1, 1, 3, 2, 1,
       1, 3, 1, 1, 1, 1, 1, 4, 1, 1, 4, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2,
       3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 2, 1, 1, 1, 1, 2, 2, 4, 1, 1,
       1, 1, 3, 2, 1, 1, 1, 1, 1, 3, 2, 1, 1, 3, 4, 1, 1, 1, 3, 1, 1, 1,
       1, 1, 3, 1, 1, 1, 1, 1, 4, 1, 1, 1, 3, 1, 2, 2, 1, 3, 2, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 4, 1, 3, 1, 4, 1, 1, 2, 3, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1,
       1, 4, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 3,
       2, 2, 1, 3, 1, 3, 3, 3, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 4, 1, 1, 1, 1,
       2, 1])

In [68]:
# COMPLETE THE CODE IN THIS CELL
submission = pd.DataFrame({'id': test['id'], 'category': pred})
submission['category'] = submission['category'].astype('int64')

In [69]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [70]:
submission_number = 0

In [71]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
# submission_number = 0

submission.to_csv(f'submission{submission_number}.csv', index=False)
submission_number += 1

## Challenge

You're trying to achieve a minimum of 75% Accuracy on your model.

## 2. Add Latent Semantic Indexing to pipeline (Learn)
<a id="p2"></a>

### Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### 2.1 Define Pipeline Components

Nest pipelines to perform SVD on our vectorization (LSA)

In [72]:
# COMPLETE THE CODE IN THIS CELL
# Transforming our Vectorization with SVD is how LSA generates topic columns
svd = TruncatedSVD(
    algorithm='randomized',
    n_iter=10
)

# vectorizer and classifier like before
vect = TfidfVectorizer(
    stop_words='english', tokenizer=None
)
clf = XGBClassifier()

# LSA pipeline with vectorizer & truncated SVD
lsa = Pipeline([
    ('vect', vect),
    ('svd', svd)
])

# combine LSA pipeline together with classifier
pipe = Pipeline([('lsa', lsa), ('clf', clf)])

### 2.2 Define Your grid search space and run a grid search with cross-validation
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [73]:
# COMPLETE THE CODE IN THIS CELL
parameters = {
    'lsa__svd__n_components': [x for x in range(2, 7, 2)], # 6
    'lsa__vect__max_df': [x/100 for x in range(50, 101, 25)], # .75
    'clf__max_depth': [x for x in range(5, 26, 5)] # 5
}

grid_search = GridSearchCV(pipe,parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warn

[17:07:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:07:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:07:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:07:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   35.9s


[17:08:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:08:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:08:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:08:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  3.2min finished
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:11:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('lsa',
                                        Pipeline(steps=[('vect',
                                                         TfidfVectorizer(stop_words='english')),
                                                        ('svd',
                                                         TruncatedSVD(n_iter=10))])),
                                       ('clf',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      gamma=None, gpu_id=None,
                                                      importance_type='gain',
                                               

In [74]:
grid_search.best_score_

0.8464810518174787

In [75]:
grid_search.best_params_

{'clf__max_depth': 5, 'lsa__svd__n_components': 6, 'lsa__vect__max_df': 0.75}

### 2.3 Make a Submission File

In [76]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [78]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [79]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [80]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{submission_number}.csv', index=False)
submission_number +=1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# 3. Add Spacy Word Embeddings
<a id="p3"></a>

### 3.1 Process the data set with spacy

In [81]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [82]:
# Continue Word Embedding Work Here
nlp = spacy.load("en_core_web_lg")

def get_word_vectors(docs):
    # YOUR CODE HERE
    return [nlp(doc).vector for doc in docs]

X_train_emb = get_word_vectors(train['description'])
X_test_emb = get_word_vectors(test['description'])

In [83]:
rfc = RandomForestClassifier(oob_score=True)

rfc.fit(X_train_emb, y)

RandomForestClassifier(oob_score=True)

In [84]:
# massively overfit with the Random Forest
print('Training Accuracy: ', rfc.score(X_train_emb, y))

Training Accuracy:  1.0


Here we use oob_score_ (out-of-bag score) as a **proxy** for the test score;<br>
for your submission, you will predict on the test set, as before

In [85]:
# validation looks decent without any tuning

rfc.oob_score_

0.7494199535962877

### 3.2 Make a Submission File

In [87]:
# YOUR CODE HERE
pred = rfc.predict(X_test_emb)

In [88]:
submission = pd.DataFrame({'id': test['id'], 'category': pred})
submission['category'] = submission['category'].astype('int64')

In [89]:
submission.to_csv(f'submission{submission_number}.csv', index=False)
submission_number +=1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# 4. Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 75% accuracy on the Kaggle competition. <br>
Once you have achieved that goal, please explore a few of the following topics: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do you create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

3. Research Singular Value Decomposition (SVD), one of the most important and powerful methods in Applied Mathematics and in all of Machine Learning.  Principal Components Analysis (PCA) -- which we used in Module 2 -- is closely releated to SVD.<br>

* [Daniela Witten](https://www.danielawitten.com/), a Professor of Mathematical Statistics at the University of Washington, recently penned a highly amusing and informative [tweetstorm](https://twitter.com/WomenInStat/status/1285611042446413824) about SVD, well worth reading!<br>
* [Stanford University Lecture on SVD](https://www.youtube.com/watch?v=P5mlg91as1c) <br>
* [StatQuest Principal Components Analysis](https://www.youtube.com/watch?v=FgakZw6K1QQ)<br>
* [Luis Serrano Principal Components Analysis](https://www.youtube.com/watch?v=g-Hb26agBFg)<br>



# Sentiment Analysis
Sentiment analysis is the use of natural language processing to systematically define subjective information.

# Documentation Classification vs Sentimental Analysis
Yes, they are different, Documentation Classification by definition classifying documents, however this does not necessarily mean algorithmically. You can classify documents manually. Sentimental Analysis is Documentation Classification except using an algorithm to do it.

# Common Applications of Sentiment Analysis
* Monitoring Social Media
* Automatic Customer Support
* Market Research